In [ ]:
import pandas as pd
from datetime import timedelta

In [ ]:
# setear parametros/variables
p_meses = [
    "enero", "febrero", "marzo", 
    "abril","mayo", "junio", 
    "julio", "agosto", "setiembre", 
    "octubre", "noviembre", "diciembre"
    ]
p_anio = "2020"
csv_columns = ['PRIMARIO', 'FUERA DE SERVICIO', 'REPOSICION', 'TIPO DE EVENTO', 'ESTADO DEL CLIMA', 'OBSERVACION']
duracion_horas_recierre = 2 # numero arbitrario de horas a asignar a las reposiciones indicadas como "recierre"

In [ ]:
# Considerar Recierre como 2 horas
def transform_recierre(fuera_servicio, reposicion):
    if reposicion.lower().strip() != "recierre":
        return reposicion
    
    reposicion = pd.to_datetime(fuera_servicio, format="%d/%m/%Y %H:%M", errors='coerce') + timedelta(hours=duracion_horas_recierre)
    return reposicion.strftime("%d/%m/%Y %H:%M")

In [ ]:
def normalizar_barrios(df1):
    # Normalizar nombres de barrios
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("\r", " "))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("\n", " "))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("  ", " "))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.strip())
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Hiódromo", "Hipódromo"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Madame Lych", "Madame Lynch"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Dr. Gaspar R. de Francia", "Dr. Gaspar Rodríguez de Francia"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Rodriguez", "Rodríguez"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mcal. Francisco.S. López", "Mcal. Francisco S. López"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace('Mcal F. S. López', "Mcal. Francisco S. López"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace('Francisco.', 'Francisco'))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mbocayty", "Mbocayaty"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mobocayaty", "Mbocayaty"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Cañada de Ybyray", "Cañada Ybyray"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Nazaret", "Nazareth"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Nazarethh", "Nazareth"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Carlos. A. López", "Carlos A. López"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Petirosi", "Pettirossi"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Cristobal", "Cristóbal"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Santisima", "Santísima"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mocayaty", "Mbocayaty"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mcal. José F. Estigarribia","Mcal. Estigarribia"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("ybyray", "Ybyray"))
    df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.rstrip("."))
    return df1


In [ ]:
for m, p_mes in enumerate(p_meses):
    input_file = f"{p_anio}{m+1:02d}"
    
    # leer archivos
    df1 = pd.read_csv(fr"../data/{input_file}.csv", encoding="ISO-8859-1")
    
    # Transformar datasets
    df1n = df1.BARRIOS.str.rstrip(',').str.rstrip(',.').str.split(',').apply(pd.Series)  # break df.BARRIOS into columns
    df1n.index = df1.set_index(csv_columns).index
    df1 = df1n.stack().reset_index(csv_columns)  # stack and reset_index
    df1.columns = csv_columns  + ['BARRIO']
    
    # Lidiar con "recierre"
    df1['REPOSICION'] = df1.apply(lambda row: transform_recierre(row['FUERA DE SERVICIO'], row['REPOSICION']), axis=1)
    
    # Agregar DURACION
    df1['DURACION'] = df1[["FUERA DE SERVICIO", "REPOSICION"]].apply(lambda x: (pd.to_datetime(x[1], format="%d/%m/%Y %H:%M", errors='coerce') - pd.to_datetime(x[0], format="%d/%m/%Y %H:%M", errors='coerce')), axis=1)
    
    # Agregar Duración en Segundos
    df1['DURACION SEGS'] = df1['DURACION'].apply(lambda x: x.seconds)
    
    # Normalizar PRIMARIO
    df1['PRIMARIO'] = df1['PRIMARIO'].apply(lambda x: x.rstrip("."))
    
    # 
    df1['OBSERVACION'] = df1['OBSERVACION'].apply(lambda x: str(x).replace("\r", " "))
    df1['OBSERVACION'] = df1['OBSERVACION'].apply(lambda x: str(x).replace("\n", " "))
    
    df1 = normalizar_barrios(df1)
    
    ## Verificar datos
    unique_barrios = sorted(list(df1["BARRIO"].unique()))
    # debe haber 69 o 68 barrios como max
    print(f"{input_file} produjo más de 69 barrios") if len(aux)>69 else None
    
    # Revisar que no queden recierres
    filas_recierre = df1.loc[(df1["REPOSICION"]=="Recierre")]
    if len(filas_recierre)>0:
        raise(f"{input_file} tiene filas con 'recierre' en columna REPOSICION")
    
    # No deben quedar barrios vacios
    if len(df1.loc[df1.BARRIO == ""]) > 0:
        raise(f"{input_file} tiene filas con BARRIOS vacios")
    
    # Exportar dataframe transformado
    df1.to_csv(f"../transformed/t_{p_mes}.csv", encoding="ISO-8859-1", index=False)
    print(f"Datos guardados en ../transformed/t_{p_mes}.csv")

    

# Verificar datos
Las celdas de abajo pueden ayudar a visualizar/testear algunas transformaciones en el dataframe

In [ ]:
# no deben repetirse barrios
aux = sorted(list(df1["BARRIO"].unique()))
aux

In [ ]:
aux4 = df1.sort_values(by=['BARRIO'], ascending=False)
aux4.loc[aux4.BARRIO == ""]

In [ ]:
AUX = df1.groupby(['TIPO DE EVENTO', 'BARRIO'])['BARRIO'].count()

In [ ]:
AUX[["Accidental"]].sort_values(ascending=False)

In [ ]:
AUX[["Programado"]].sort_values(ascending=False)

In [ ]:
aux3 = df1[['BARRIO', 'DURACION']]

In [ ]:
aux4 = df1.sort_values(by=['DURACION'], ascending=False)

In [ ]:
aux4.sort_values(by=["DURACION"], ascending=False)

In [ ]:
aux4.loc[(aux4["REPOSICIONN"]=="Recierre") & (aux4["FUERA DE SERVICIO"].str.contains("10/01/2020"))]

In [ ]:
aux4.loc[(aux4["FUERA DE SERVICIO"].str.contains("31/01/2020"))]

In [ ]:
tdelta2 = pd.to_datetime("7/01/2020 01:12", format="%d/%m/%Y %H:%M") - pd.to_datetime("6/01/2020 23:54", format="%d/%m/%Y %H:%M")
tdelta2

In [ ]:
pd.to_datetime("7/01/2020 01:12", format="%d/%m/%Y %H:%M")